<a href="https://colab.research.google.com/github/gywnsrkd/2022_IISE/blob/main/trial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install
!pip install -q roboflow

import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


Cloning into 'yolov5'...
remote: Enumerating objects: 12148, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 12148 (delta 0), reused 0 (delta 0), pack-reused 12142
Receiving objects: 100% (12148/12148), 11.93 MiB | 26.22 MiB/s, done.
Resolving deltas: 100% (8408/8408), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 7.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 145 kB 7.4 MB/s 
     |████████████████████████████████| 178 kB 46.7 MB/s 
     |████████████████████████████████| 1.1 MB 49.3 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 138 kB 52.5 MB/

In [3]:
from zipfile import ZipFile
import os

def unzip(source_file, dest_path):
    with ZipFile(source_file, 'r') as zf:
        zipInfo = zf.infolist()
        for member in zipInfo:
            try:
                member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                zf.extract(member, dest_path)
            except:
                raise Exception('what?!')

In [4]:
file_names = '/content/gdrive/MyDrive/cap/ex3.v2i.yolov5pytorch.zip'
unzip(file_names,'/content/yolov5/custom_dataset/')

In [5]:
%cat /content/yolov5/custom_dataset/data.yaml

train: ../train/images
val: ../valid/images

nc: 2
names: ['dangerous', 'safe']

In [6]:
import yaml

with open('/content/yolov5/custom_dataset/data.yaml', 'r') as f:
  data = yaml.load(f, Loader=yaml.FullLoader)

print(data)

data['train'] = "/content/yolov5/custom_dataset/train"
data['val'] = "/content/yolov5/custom_dataset/valid"

with open('/content/yolov5/custom_dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 2, 'names': ['dangerous', 'safe']}
{'train': '/content/yolov5/custom_dataset/train', 'val': '/content/yolov5/custom_dataset/valid', 'nc': 2, 'names': ['dangerous', 'safe']}


In [8]:
!python train.py --img 640 --batch 64 --epochs 50 --data /content/yolov5/custom_dataset/data.yaml --weights yolov5s.pt      

train: weights=yolov5s.pt, cfg=, data=/content/yolov5/custom_dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-241-g7d87b9e Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hs

In [9]:
save_path = '/content/gdrive/MyDrive/cap/result/효준'
save_file_path = '/content/yolov5/runs/train/exp2'

shutil.move(save_file_path, save_path)

'/content/gdrive/MyDrive/cap/result/효준'

In [10]:
!python detect.py --source '/content/gdrive/MyDrive/cap/맹견영상.mp4' --weight '/content/gdrive/MyDrive/cap/result/효준/weights/best.pt'

detect: weights=['/content/gdrive/MyDrive/cap/result/효준/weights/best.pt'], source=/content/gdrive/MyDrive/cap/맹견영상.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-241-g7d87b9e Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/470) /content/gdrive/MyDrive/cap/맹견영상.mp4: 384x640 1 dangerous, Done. (0.016s)
video 1/1 (2/470) /content/gdrive/MyDrive/cap/맹견영상.mp4: 384x640 1 dangerous, Done. (0.011s)
video 1/1 (3/470) /content/gdrive/MyDrive/cap/맹견영상.mp4: 384x640 1 dangerous, Done. (0.010

In [11]:
save_path = '/content/gdrive/MyDrive/cap/result/효준/영상'
save_file_path = '/content/yolov5/runs/detect/exp'

shutil.move(save_file_path, save_path)

'/content/gdrive/MyDrive/cap/result/효준/영상'

In [12]:
!python detect.py --source '/content/gdrive/MyDrive/cap/테스트영상.mp4' --weight '/content/gdrive/MyDrive/cap/result/효준/weights/best.pt'

detect: weights=['/content/gdrive/MyDrive/cap/result/효준/weights/best.pt'], source=/content/gdrive/MyDrive/cap/테스트영상.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-241-g7d87b9e Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/451) /content/gdrive/MyDrive/cap/테스트영상.mp4: 640x384 1 safe, Done. (0.017s)
video 1/1 (2/451) /content/gdrive/MyDrive/cap/테스트영상.mp4: 640x384 1 safe, Done. (0.010s)
video 1/1 (3/451) /content/gdrive/MyDrive/cap/테스트영상.mp4: 640x384 Done. (0.011s)
video 1/1 (4/451) /c

In [13]:
save_path = '/content/gdrive/MyDrive/cap/result/효준/영상/'
save_file_path = '/content/yolov5/runs/detect/exp'

shutil.move(save_file_path, save_path)

'/content/gdrive/MyDrive/cap/result/효준/영상/exp'